<a href="https://colab.research.google.com/github/prashant-chauhan11/machine_learning/blob/master/housing/housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [0]:
df = pd.read_csv("housing_train.csv")

In [0]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea
0,Brunswick,52 Evans St,3,h,1650000,S,Nelson,5.2,3056,3.0,1.0,2.0,495.0,141.0,1920.0,Moreland
1,Reservoir,85 Radford Rd,5,h,791000,S,Ray,11.2,3073,4.0,3.0,1.0,961.0,NaN,NaN,Darebin
2,Newport,99 Anderson St,3,h,785000,S,RT,8.4,3015,3.0,1.0,1.0,185.0,NaN,NaN,Hobsons Bay
3,Brighton East,4/377 South Rd,2,u,755000,SP,Buxton,10.7,3187,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hawthorn East,3 Jaques St,5,h,2500000,VB,RT,7.5,3123,5.0,3.0,3.0,757.0,240.0,1925.0,Boroondara


In [0]:
df.nunique()

Suburb           142
Address         7468
Rooms              9
Type               3
Price           1690
Method             5
SellerG          182
Distance          93
Postcode          94
Bedroom2          10
Bathroom           9
Car                9
Landsize        1100
BuildingArea     386
YearBuilt        131
CouncilArea       19
dtype: int64

In [0]:
enc = LabelEncoder()
df['Suburb'] = enc.fit_transform(df['Suburb'].astype(str))
df['SellerG'] = enc.fit_transform(df['SellerG'].astype(str))
df['CouncilArea'] = enc.fit_transform(df['CouncilArea'].astype(str))

In [0]:
df.drop(['Address', 'Postcode', 'CouncilArea', 'SellerG', 'Landsize'], axis = 1, inplace = True)

In [0]:
df.dtypes

Suburb            int64
Rooms             int64
Type             object
Price             int64
Method           object
Distance        float64
Bedroom2        float64
Bathroom        float64
Car             float64
BuildingArea    float64
YearBuilt       float64
dtype: object

In [0]:
df.isnull().sum()

Suburb             0
Rooms              0
Type               0
Price              0
Method             0
Distance           0
Bedroom2        1559
Bathroom        1559
Car             1559
BuildingArea    4209
YearBuilt       3717
dtype: int64

In [0]:
'''df = df[~df['Bedroom2'].isnull()]
df = df[~df['Landsize'].isnull()]
df.reset_index(drop = True)'''

df.loc[df['Bedroom2'].isnull(), 'Bedroom2'] = df['Bedroom2'].median()
#df.loc[df['Landsize'].isnull(), 'Landsize'] = df['Landsize'].mean()
df.loc[df['Bathroom'].isnull(), 'Bathroom'] = df['Bathroom'].median()
df.loc[df['Car'].isnull(), 'Car'] = df['Car'].median()
df.loc[df['BuildingArea'].isnull(), 'BuildingArea'] = df['BuildingArea'].median()
df.loc[df['YearBuilt'].isnull(), 'YearBuilt'] = df['YearBuilt'].median()

In [0]:
df.isnull().sum()

Suburb          0
Rooms           0
Type            0
Price           0
Method          0
Distance        0
Bedroom2        0
Bathroom        0
Car             0
BuildingArea    0
YearBuilt       0
dtype: int64

In [0]:
obj_column = df.select_dtypes(['object']).columns
obj_column

Index(['Type', 'Method'], dtype='object')

In [0]:
for col in obj_column:
  k = df[col].unique()
  for cat in k:
    name = str(col)+'_'+str(cat)
    df[name] = (df[col]==cat).astype(int)
  del df[col]

In [0]:
df.shape

(7536, 17)

In [0]:
X, Y = df.drop(['Price'], axis = 1), df["Price"]

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 1)

In [0]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(6028, 16)
(6028,)
(1508, 16)
(1508,)


In [0]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
from sklearn.metrics import mean_absolute_error

In [0]:
pred = model.predict(X_test)
mae = mean_absolute_error(Y_test, pred)
mae

304533.59133185877

In [0]:
score = model.score(X_test, Y_test)
score

0.5373453058661001

In [0]:
for i in zip(model.coef_, X_train.columns):
  print(i)

(-1797.5041112196168, 'Suburb')
(283661.4041679318, 'Rooms')
(-46485.09967889511, 'Distance')
(-53347.185389476246, 'Bedroom2')
(195552.4706619714, 'Bathroom')
(40315.480806558306, 'Car')
(918.8094186740927, 'BuildingArea')
(-2577.625315327423, 'YearBuilt')
(165150.79480458915, 'Type_h')
(-125748.92780198134, 'Type_u')
(-39401.86700260776, 'Type_t')
(31583.346641383494, 'Method_S')
(-89911.571481616, 'Method_SP')
(3240.7807008640116, 'Method_VB')
(-55132.29327816835, 'Method_PI')
(110219.7374175367, 'Method_SA')


**Decision Tree**

In [0]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [0]:
model = DecisionTreeRegressor()

model.fit(X_train, Y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [0]:
pred = model.predict(X_test)

In [0]:
mean_absolute_error(Y_test, pred)

278451.54876118194

In [0]:
print(model.score(X_test, Y_test))

0.4840319050146969


In [0]:
params = {
    'max_depth' : [10, 20, 30],
    'min_samples_leaf':[9],
    'min_samples_split':[2, 3, 4, 5, 10, 20]
}

In [0]:
random_search = RandomizedSearchCV(model, cv = 10, param_distributions = params, 
                                   scoring = 'neg_mean_absolute_error', n_iter = 10)

In [0]:
random_search.fit(X_train, Y_train)

RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=DecisionTreeRegressor(criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort=False,
                                                   random_state=None,
                                                   splitter='best'),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_d

In [0]:
random_search.best_estimator_

DecisionTreeRegressor(criterion='mse', max_depth=30, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=9,
                      min_samples_split=5, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [0]:
best = random_search.best_estimator_
print(mean_absolute_error(Y_test, best.predict(X_test)))
best.score(X_test, Y_test)*100

242256.77346448


65.3427136236294

0.14291739512316148 0.3251308428080916 0.23114043590994054 0.0013481849274908772 0.04297325175919998 0.00948620815602848 0.09386664365775359 0.029254394615203794 0.11600114684862148 0.0002027082652804883 0.0008446216451012081 0.0053496345430375175 0.0012602248320943805 8.753156194324507e-05 0.00013677534705150248 0.0


**Random Forest**

In [0]:
from sklearn.ensemble import RandomForestRegressor

In [0]:
param_dist = {"n_estimators":[50, 100, 200, 300, 500, 600, 700],
              "max_features": [1, 2, 3, 4, 5, 10, 13, 16],
              "bootstrap": [ False], 
                'criterion':['mse'],
                'max_depth':[None,5,30, 40, 50],
                'min_samples_leaf':[2, 5,20], 
                'min_samples_split':[2,10]
                  }

In [0]:
model = RandomForestRegressor()

random_search = RandomizedSearchCV(model, param_distributions=param_dist,
                                   n_iter=10,scoring='neg_mean_absolute_error',cv=5, verbose=1)
random_search.fit(X_train, Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.9min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...


In [0]:
random_search.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=40,
                      max_features=5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=700,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [0]:
best = random_search.best_estimator_
print(mean_absolute_error(Y_test, best.predict(X_test)))
best.score(X_test, Y_test)*100

213776.24752281318


74.9088351365285

XGB

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost.sklearn import XGBRegressor

In [0]:
gbm_params={'n_estimators':[10,50,100, 200, 300, 400],
           'learning_rate': [0.1 ,0.2, 0.4, 0.6, 0.8,1],
            'max_depth':[1,2,3,4,5,6, 7, 8],
           'min_samples_split':[2,5,10,20, 30, 40, 50, 60],
             'min_samples_leaf':[2,5,10,20, 30, 40, 50, 60],
            'subsample':[0.1,0.5,0.8,1],
            'max_features':[1, 3, 5, 7, 8, 9, 10, 12, 14, 16]
           }

In [0]:
gbm = GradientBoostingRegressor()

In [0]:
random_search = RandomizedSearchCV(gbm, param_distributions=gbm_params, cv = 5, n_iter=10, n_jobs = -1, scoring='neg_mean_absolute_error')

In [0]:
random_search.fit(X_train, Y_train)

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=GradientBoostingRegressor(alpha=0.9,
                                                       criterion='friedman_mse',
                                                       init=None,
                                                       learning_rate=0.1,
                                                       loss='ls', max_depth=3,
                                                       max_features=None,
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                            

In [0]:
random_search.best_estimator_

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=7,
                          max_features=16, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=2, min_samples_split=40,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
best = random_search.best_estimator_
print(mean_absolute_error(Y_test, best.predict(X_test)))
best.score(X_test, Y_test)*100

194366.0507889061


78.62795903357154

In [0]:
best.feature_importances_

array([0.16285719, 0.24706394, 0.20334577, 0.01382632, 0.05083713,
       0.02131148, 0.12400212, 0.05679688, 0.08153061, 0.0191629 ,
       0.00111799, 0.0068942 , 0.00142634, 0.00554322, 0.00253601,
       0.0017479 ])

In [0]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.2,0.3,0.4,0.5],
                "gamma":[i/10.0 for i in range(0,10)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000, 1100, 1200]
             }

In [0]:
xgb = XGBRegressor()

In [0]:
random_search = RandomizedSearchCV(xgb, n_jobs = -1, cv = 5, param_distributions=xgb_params, n_iter=10, scoring='neg_mean_absolute_error')

In [0]:
random_search.fit(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:19:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_st...
                                        'max_depth': [2, 3, 4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 5, 10],
                                      

In [0]:
random_search.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0.1,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=1, reg_lambda=1e-05, scale_pos_weight=2, seed=None,
             silent=None, subsample=0.7, verbosity=1)

In [0]:
best = random_search.best_estimator_
print(mean_absolute_error(Y_test, best.predict(X_test)))
best.score(X_test, Y_test)*100

187571.1743306532


80.23610769284743